# モジュールのインストール

In [1]:
!pip install selenium
!pip install webdriver_manager
!pip install requests
!pip install BeautifulSoup4
!pip install chardet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# ページごとにレビューの取得する関数

In [3]:
def get_all_review(soup, dataList):
    # 全てのレビューの取得
    reviews = soup.find_all('div', attrs={'class':'item-listContents'})

    for review in reviews:
        # 個々のレビューに対してデータを取得
        if review.find('p', attrs={'class': 'item-title'}) is not None:
            title = review.find('p', attrs={'class': 'item-title'}).text
        else:
            title = ''
            
        if review.find('span', attrs={'class': 'reviewPoint'}) is not None:
            point = review.find('span', attrs={'class': 'reviewPoint'}).text
        else:
            point = ''
        
        if review.find('ul', attrs={'class':'iconCompanionTag'}) is not None:
            tag = review.find('ul', attrs={'class':'iconCompanionTag'}).text.replace('\n', '')
        else:
            tag = ''

        if review.find('div', attrs={'class': 'item-reviewTextInner'}) is not None:
            text = review.find('div', attrs={'class': 'item-reviewTextInner'}).text
        else:
            text = ''
        
        if review.find_all('ul', attrs={'class': 'item-reviewDetail'}) is not None:
            details = review.find_all('ul', attrs={'class': 'item-reviewDetail'})
        else:
            details = ''

        # detailsの中身を今後色々しやすいように辞書形式に変更  
        detailList = []
        detailDict = {}
        for ul_tag in details:
            for li in ul_tag.find_all('li'):
                detailList.append(li.text)

        for item in detailList:
            item = item.split("：")
            k, v = item[0], item[1]
            detailDict[k] = v

        dtum = {'title': title, 'point': point, 'tag':tag, 'text': text, 'details': detailDict}
        dataList.append(dtum)
        time.sleep(1)

# 全件取得を試みる

## ハイパーパラメーターの設定

In [4]:
# url = "https://www.jalan.net/kankou/spt_06203cc3320040667/kuchikomi/?screenId=OUW3701&influxKbn=0"
# 鶴岡市加茂水族館
# url = "https://www.jalan.net/kankou/spt_06203cc3320040667/kuchikomi/"
# zao
# url = "https://www.jalan.net/kankou/spt_06201cd2110077451/kuchikomi/"
# 羽黒山五重塔(haguro)
# url = "https://www.jalan.net/kankou/spt_06424ae2180021220/kuchikomi/"
# 羽黒山(mtHaguro)
# url = "https://www.jalan.net/kankou/spt_06424ab2010001853/kuchikomi/"
# 蔵王温泉大露天風呂（zaoHotspring）
# url = "https://www.jalan.net/kankou/spt_guide000000152068/kuchikomi/"
# 銀山温泉　（ginzan）
# url = "https://www.jalan.net/kankou/spt_06212cd2110077654/kuchikomi/"
urls = ["https://www.jalan.net/kankou/spt_guide000000161658/kuchikomi/", 
        "https://www.jalan.net/kankou/spt_05403ab2010002816/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06202cd2110076440/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06201ae3292019427/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06202ag2130014109/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06204ae2180022287/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06212ag2130014710/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06203cd2110077904/kuchikomi/",
        "https://www.jalan.net/kankou/spt_guide000000186569/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06441cd2110078399/kuchikomi/",
        "https://www.jalan.net/kankou/spt_guide000000173405/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06201ee4570112037/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06207cd2110078131/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06441ag2130009280/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06365cd2110077795/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06323ag2130015562/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06213ag2130012875/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06210cd2110077452/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06201ah3330040923/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06203ab2012019468/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06421ab2040005453/kuchikomi/",
        "https://www.jalan.net/kankou/spt_06427ag2130015952/kuchikomi/"]

filenames = ["guttoYamagata", "mtTyokai", "onogawaHotspring", "bunsyokan", "uesgiShrine", "yamaKyo", "shuwaShrine", 
             "yunohamaHotspring", "showaKan", "atsumiHotspring", "totoroTree", "zaoRopeway", "kaminoyamaHotspring", 
             "itsukusimaShrine", "hizioriHotspring", "hakusanShrine", "kumanoShrine", "tendoHotspring", "kazyoPark",
            "mtGassyan", "mogamiRiver", "yudonoShrine"]

count = 0

In [6]:
for (url, filename) in zip(urls, filenames):
# スクレイピングブロック
    dataList = []
    startTime = time.time()
    # 一度ブラウザ起動
    options = Options()
    options.add_argument('--headless')
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    browser.get(url)
    print(startTime)
    while True:
        url = browser.current_url
        try:
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            get_all_review(soup, dataList)
            browser.find_element_by_link_text('次へ').click()
            if count % 10 == 0:
                print("pages:{0}".format(count))
                print('==============================================')
            count += 1
        except NoSuchElementException:
            browser.quit()
            break
    
# データの加工ブロック
    reviews = []

    for review in dataList:
        reviewDict = {'title': review['title'], 'point':review['point'], 'tag':review['tag'], 'text':review['text']}
        if '行った時期' in review['details']:  
            reviewDict['行った時期'] = review['details']['行った時期']
        if '混雑具合' in review['details']:
            reviewDict['混雑具合'] = review['details']['混雑具合']
        if '滞在時間' in review['details']:
            reviewDict['滞在時間'] = review['details']['滞在時間']
        if '人数' in review['details']:
            reviewDict['人数'] = review['details']['人数']
        if '投稿日' in review['details']:
            reviewDict['投稿日'] = review['details']['投稿日']

        reviews.append(reviewDict)
#   レビュー数の確認
    print(len(reviews))

# csvに加工するブロック
    df = pd.DataFrame(reviews)
    #カラムの並び替え
    df.reindex(columns=['postDay', 'point', ' when', 'congestion', 'tag', 'numberOfPeople', 'title', 'text'])
    df.head().append(df.tail())
    df.to_csv('../python_nlp_yamagata/{filename}.csv'.format(filename = filename))
    elapsed_time = time.time() - startTime
    print(elapsed_time)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


1623375846.243903
pages:0
pages:10
pages:20
pages:30
pages:40




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472


508
571.9050459861755


Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


...

1623382326.117027
pages:570




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


117
135.18261790275574
1623382461.299712
pages:580
pages:590




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kazu/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


157
176.08059120178223
1623382637.380379
pages:600
120
135.595458984375


In [ ]:
# i = 0
# # 山寺
# # url = "https://www.jalan.net/kankou/spt_guide000000152086/kuchikomi/"
# # 鶴岡市加茂水族館
# # url = "https://www.jalan.net/kankou/spt_06203cc3320040667/kuchikomi/"


# dataList = []
# startTime = time.time()
# print(startTime)
# while True:
#     url = browser.current_url
#     try:
#         t = time.time()
#         res = requests.get(url)
#         soup = BeautifulSoup(res.content, 'html.parser')
#         get_all_review(soup, dataList)
#         browser.find_element_by_link_text('次へ').click()
#         iteratorTime = time.time() - t
#         print("pages:{0}\niteratorTime:{1}".format(i, iteratorTime))
#         print('==============================================')
#         i += 1
#     except NoSuchElementException:
#         browser.quit()
#         break
# elapsed_time = time.time() - startTime
# print(elapsed_time)